### Transmission and Tunneling in TMM* method
*TMM : Transfer Matrix Method


In [ ]:
import numpy as np
import numpy.linalg as la
import math
from scipy.signal import find_peaks
from joblib import Parallel, delayed
from scipy.constants import physical_constants
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact_manual

###############################################################################
# Global Constants and Unit Conversions
###############################################################################
# 1 Bohr = 0.52917721067 Angstrom, so 1 Angstrom = 1/0.52917721067 Bohr ~ 1.88973
bohr_radius_ang = physical_constants['Bohr radius'][0] * 1e10  # in Angstrom
ang2bohr = 1 / bohr_radius_ang  # ~1.88973

# Hartree energy in eV
har2ev = physical_constants['Hartree energy'][0] / physical_constants['electron volt'][0]  # ~27.21140795

###############################################################################
# 1. Basic Helpers & Potential Construction
###############################################################################
def make_open_kp_potential(num_barriers: int,
                           barrier_height: float,
                           barrier_width: float,
                           well_width: float,
                           dx: float,
                           bias_eV: float = 0.0):
    """
    Create a 1D potential array for a finite chain of 'num_barriers' barriers,
    each barrier of given width and height, separated by wells (V=0).
    The overall domain spans from 0 to (left + num_barriers*(barrier_width+well_width) + right),
    where the leftmost and rightmost margins have 0 potential.
    
    A triangular bias is applied only in the region containing the barrier structure 
    (from x = left to x = left + num_barriers*(barrier_width+well_width)).
    The bias is 0 at the boundaries and reaches bias_eV at the midpoint.
    """
    left = well_width + 2  # 2 for electrode
    right = 0 + 2

    L_segment = barrier_width + well_width
    L_barrier_total = num_barriers * L_segment
    L_total = left + L_barrier_total + right
    Nx = int(np.ceil(L_total / dx))
    x = np.linspace(0, L_total, Nx)
    
    V = np.zeros(Nx)
    # Build the periodic potential in the barrier region
    for b in range(num_barriers):
        start_x = left + b * L_segment 
        end_x   = left + b * L_segment + barrier_width
        start_idx = int(start_x / dx)
        end_idx   = int(end_x / dx)
        V[start_idx:end_idx] = barrier_height
    
    # Apply a triangular bias over the barrier region
    if abs(bias_eV) > 1e-12:
        bias = np.zeros(Nx)
        x0 = left - (well_width)
        x1 = left + L_barrier_total
        for i, xi in enumerate(x):
            if xi < x0:
                bias[i] = 0
            elif xi > x1:
                bias[i] = bias_eV
            else:
                bias[i] = bias_eV * ((xi - x0) / (x1 - x0))
        V = V + bias
    return x, V

def k_local(E, V):
    """
    Local wave number for E - V in 1D (atomic units: ħ=1, m=1).
      k = sqrt(2*(E - V)), real if E>V, imaginary if E<V.
    """
    delta = E - V
    if delta >= 0:
        return np.sqrt(2*delta + 0j)
    else:
        return 1j * np.sqrt(-2*delta)

def interface_matrix(k1, k2):
    """
    1D interface matrix between regions with wave numbers k1 and k2.
    """
    r = k2 / k1
    return 0.5 * np.array([[1 + r, 1 - r],
                           [1 - r, 1 + r]], dtype=complex)

def full_transfer_matrix_open(x, V, E, dx):
    """
    Compute the full transfer matrix for an open-boundary potential V(x)
    by stepping through the grid cell-by-cell.
    """
    
    Nx = len(x)
    M_total = np.eye(2, dtype=complex)
    
    k_left  = k_local(E, 0.0)
    k_right = k_local(E, 0.0)
    k_layer0 = k_local(E, V[0])
    
    M_int_left = interface_matrix(k_left, k_layer0)
    M_total = M_int_left @ M_total
    
    for i in range(Nx - 1):
        k_i     = k_local(E, V[i])
        k_ip1   = k_local(E, V[i+1])
        M_prop  = np.array([[np.exp(1j*k_i*dx),  0],
                           [0, np.exp(-1j*k_i*dx)]], dtype=complex)
        M_total = M_prop @ M_total
        M_int = interface_matrix(k_i, k_ip1)
        M_total = M_int @ M_total
    
    k_last = k_local(E, V[-1])
    M_prop_last = np.array([[np.exp(1j*k_last*dx),  0],
                            [0, np.exp(-1j*k_last*dx)]], dtype=complex)
    M_total = M_prop_last @ M_total
    M_int_right = interface_matrix(k_last, k_right)
    M_total = M_int_right @ M_total
    
    return M_total

def transmission_coefficient_open(M):
    """
    Compute transmission coefficient T = 1 / |M[0,0]|^2.
    """
    return 1.0 / np.abs(M[0,0])**2

###############################################################################
# 2. Compute Transmission and Find Resonance Peaks (Parallelized)
###############################################################################
def compute_transmission_peaks(num_barriers: int,
                               barrier_height: float,
                               barrier_width: float,
                               well_width: float,
                               dx: float,
                               E_array: np.ndarray,
                               bias_eV: float = 1.0):
    """
    For a given number of barriers, build the open-boundary potential,
    compute T(E) via TMM (in parallel), and identify resonance peaks.
    
    Returns a dict with:
      "x": coordinate array,
      "V": potential array,
      "T_tmm": transmission array (clamped to [0,1]),
      "peaks_tmm": resonance peak energies.
    """
    print("\n"*2)
    print("Prepare for calculation")
    x, V = make_open_kp_potential(num_barriers, barrier_height, barrier_width, well_width, dx,
                                  bias_eV=bias_eV)
    
    def compute_T(E):
        M = full_transfer_matrix_open(x, V, E, dx)
        T_val = transmission_coefficient_open(M)
        return max(min(T_val, 1.0), 1e-12)
    
    print("\n"*2)
    print("calculate the transmission...")
    T_tmm = Parallel(n_jobs=-1)(delayed(compute_T)(E) for E in E_array)
    T_tmm = np.array(T_tmm)
    
    logT = np.log1p(T_tmm)
    pkidx, _ = find_peaks(logT, prominence=1e-3)
    Epeaks_tmm = E_array[pkidx]

    print("\n"*2)
    print("calculation done!!!")
    
    return dict(x=x, V=V, T_tmm=T_tmm, peaks_tmm=Epeaks_tmm)

###############################################################################
# 3. Plot Using Plotly: Potential, Transmission, and Resonance Peaks vs. Occurrence
###############################################################################
def resonance_vs_occurrence(num_barriers, barrier_height, barrier_width, well_width, bias_eV):
    """
    Compute T(E) via the transfer matrix method, find resonance peaks, and plot three panels:
      (1) Potential with horizontal lines at each resonance energy.
      (2) Transmission (log scale on x-axis) vs. Energy with markers at resonances.
      (3) Scatter plot of Resonance Peak Energy vs. Occurrence (peak index).
    """
    dx = 0.05
    if bias_eV > 0:
        emin = 0
        emax = (barrier_height + bias_eV) * 3
    else:
        emin = 0.0 + bias_eV
        emax = (barrier_height) * 3

    eres = 1500
    E_array = np.linspace(emin, emax, eres)
    
    # Compute transmission using TMM (parallelized)
    result = compute_transmission_peaks(num_barriers, barrier_height, barrier_width, well_width, dx, E_array, bias_eV=bias_eV)
    x = result["x"]
    V = result["V"]
    T_tmm = result["T_tmm"]
    pk_tmm = result["peaks_tmm"]  # resonance energies
    occurrence = np.arange(1, len(pk_tmm)+1)
    
    # Generate colors for each peak using Plotly's colorscale
    if len(pk_tmm) > 1:
        colors = px.colors.sample_colorscale("Rainbow", [i/(len(pk_tmm)-1) for i in range(len(pk_tmm))])
    elif len(pk_tmm) == 1:
        colors = [px.colors.sample_colorscale("Rainbow", [0.5])[0]]
    else:
        colors = []
    
    # Create subplots with 1 row and 3 columns
    fig = make_subplots(rows=1, cols=3,)
    
    # Left panel: Potential curve and horizontal lines at resonance energies
    fig.add_trace(go.Scatter(x=x, y=V * har2ev,
                            mode='lines',
                            line=dict(color="black"),
                            name="Potential",
                            showlegend=False),
                row=1, col=1)
    # Add horizontal lines for each resonance energy
    for i, epeak in enumerate(pk_tmm):
        fig.add_shape(type="line",
                    x0=x[0], x1=x[-1],
                    y0=epeak * har2ev, y1=epeak * har2ev,
                    line=dict(color=colors[i], dash="dash"),
                    row=1, col=1)
    
    fig.update_yaxes(range=[emin * har2ev, emax * har2ev], row=1, col=1)

    # Center panel: Transmission vs. Energy (x-axis: T, y-axis: Energy), x-scale log
    fig.add_trace(go.Scatter(x=T_tmm, y=E_array * har2ev,
                            mode='lines',
                            line=dict(color="black"),
                            name="TMM",
                            showlegend=False),
                row=1, col=2)
    # Add markers at resonance peaks
    for i, epeak in enumerate(pk_tmm):
        idx = np.argmin(np.abs(E_array - epeak))
        fig.add_trace(go.Scatter(x=[T_tmm[idx]], y=[epeak * har2ev],
                                mode='markers',
                                marker=dict(color=colors[i], size=8),
                                name=f"Resonance {i+1}",
                                showlegend=False),
                    row=1, col=2)

    fig.update_layout(title_text=f"Resonance vs. Occurrence for {num_barriers} barriers", 
                    width=1200, height=500,
                    showlegend=False)

    # Add markers at resonance peaks
    for i, epeak in enumerate(pk_tmm):
        idx = np.argmin(np.abs(E_array - epeak))
        fig.add_trace(go.Scatter(x=[T_tmm[idx]], y=[epeak * har2ev],
                                 mode='markers',
                                 marker=dict(color=colors[i], size=8),
                                 name=f"Resonance {i+1}"),
                      row=1, col=2)

    fig.update_xaxes(title_text="Transmission", type="linear", range=[0, 1], row=1, col=2)
    fig.update_yaxes(title_text="Energy (eV)", range=[emin * har2ev, emax * har2ev], row=1, col=2)

    # scale secelector for Transmission (log, linear)
    fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            buttons=list([
                dict(
                    args=[{"xaxis2.type": "linear", "xaxis2.range": [0, 1]}],
                    label="Linear",
                    method="relayout"
                ),
                dict(
                    args=[{"xaxis2.type": "log", "xaxis2.range": [np.log10(1e-6), 0]}],
                    label="Log",
                    method="relayout"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.5,
            xanchor="center",
            y=1.2,
            yanchor="top"
            )
        ]
    )
    
    # Right panel: Resonance Peak Energy vs. Occurrence
    fig.add_trace(go.Scatter(x=occurrence, y=pk_tmm * har2ev,
                             mode='markers',
                             marker=dict(color=colors, size=8),
                             name="Resonance Peaks"),
                  row=1, col=3)
    fig.update_xaxes(title_text="Occurrence", range=[0, len(pk_tmm)+1], row=1, col=3)
    fig.update_yaxes(title_text="Resonance Energy (eV)", range=[emin * har2ev, emax * har2ev], row=1, col=3)
    
    fig.update_layout(title_text=f"Resonance vs. Occurrence for {num_barriers} barriers", width=1200, height=500)
    fig.show()

###############################################################################
# 4. Interactive Widget
###############################################################################

slider_layout = widgets.Layout(width='400px')  # Set a fixed width for all sliders
label_style = {'description_width': '150px'}  # Uniform label width for alignment

@interact_manual(
    num_barriers=widgets.IntSlider(min=1, max=40, step=1, value=1, 
                                    description='# of Barriers : ',
                                    style=label_style,
                                    layout=slider_layout),
    barrier_height=widgets.FloatSlider(min=0.5, max=20, step=0.1, value=3, 
                                    description='Barrier Height [eV] : ',
                                    style=label_style,
                                    layout=slider_layout),
    barrier_width=widgets.FloatSlider(min=0.1, max=500.0, step=0.1, value=3.0,
                                    description='Barrier Width [Ang] : ',
                                    style=label_style,
                                    layout=slider_layout),
    well_width=widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=3.0,
                                    description='Well Width [Ang] : ',
                                    style=label_style,
                                    layout=slider_layout),
    bias_eV=widgets.FloatSlider(min=-4, max=4, step=0.1, value=0.0,
                                    description='Linear Bias [eV] : ',
                                    style=label_style,
                                    layout=slider_layout)
)

def interactive(num_barriers, barrier_height, barrier_width, well_width, bias_eV):
    print(readme)

    resonance_vs_occurrence(num_barriers,
                              barrier_height / har2ev,
                              barrier_width * ang2bohr,
                              well_width * ang2bohr,
                              bias_eV / har2ev)

if __name__ == "__main__":
    readme = r"""
V
↑
│     V₀
│  ┌────────────────────┐           <-- V = V₀
│  │                    │
│  │                    │
│  │                    │        
│  │                    │       
│──┘                    └───────   <-- V = 0 
└────────────────────────────────→ x
   <------   a   ------><---b--->
    
    Barrier Height  = V₀ 
    Barrier width   = a
    Well width      = b
    Period of cell  = a + b
    """
    print(readme)
    pass


interactive(children=(IntSlider(value=1, description='# of Barriers : ', layout=Layout(width='400px'), max=40,…


V
↑
│     V₀
│  ┌────────────────────┐           <-- V = V₀
│  │                    │
│  │                    │
│  │                    │        
│  │                    │       
│──┘                    └───────   <-- V = 0 
└────────────────────────────────→ x
   <------   a   ------><---b--->
    
    Barrier Height  = V₀ 
    Barrier width   = a
    Well width      = b
    Period of cell  = a + b
    


## Transmission through finite barriers using Transfer Matrix Method (TMM)
#### Minsu Jeong, KAIST Electrical Engineering

###### Last updated : 2025. 03. 12
